# Utils 
## JSON

The JSON utils helps with managing JSON files. It includes the utils:

- loadJSON    : Returns a json from file, string, or python object. 
- ConfigurationToJSON/JSONToConfiguration: converts dict with unum (unit) object (e.g. a configuration file) to unum 
                                            and vice versa.

- convertJSONtoPandas : Converts JSON to pandas dataframe by flattening the path. uses processJSONToPandas as a utility function. 
- processJSONToPandas : creates dataframe from JSON where the values are int,float, or string and lists. It does not handle dicts/ lists of dicts. 
                        To do so, use convertJSONtoPandas. 
- JSONVariations      : Creates a list of variations from a base JSON file. 
- compareJSON : compares arbitrary number of JSONS and returns a pandas dataframe with the results (not implemented yet). 

### loadJSON 

Loading a json from the dist 

In [1]:
from hera.utils.jsonutils import loadJSON

In [2]:
jsonSTR = '{ "a" : 1, "b" : 2}'
loadJSON(jsonSTR)

{'a': 1, 'b': 2}

### ConfigurationToJSON/JSONToConfiguration

The ConfigurationToJSON/JSONToConfiguration converts dict with unum to string and vice versa. This is useful for configuration files. 



In [3]:
from unum.units import * 
from hera.utils.jsonutils import ConfigurationToJSON,JSONToConfiguration

configuration = {
    'a' : 5*m, 
    'b' : 1*m/s,
    'c' : [3*s,1*s,2*s]
}

ConfigurationToJSON(configuration)

{'a': '5*m', 'b': '1.0*m/s', 'c': ['3*s', '1*s', '2*s']}

Converting backwards (read the configuration from file)

In [4]:
JSONToConfiguration(configuration)

{'a': 5 [m], 'b': 1.0 [m/s], 'c': [3 [s], 1 [s], 2 [s]]}

### JSON variations

JSONVariations gets a base file and a variation file and creates variations of the base by substituting the values in the variation files. 

To use the JSONVariations first import it:

In [5]:
from hera.utils.jsonutils import JSONVariations

To use it the user must provide a base JSON file and a variation file. 

A variation file is a list of variation groups. Each variation group is a mapping between a number of [json paths](https://docs.oracle.com/cd/E60058_01/PDF/8.0.8.x/8.0.8.0.0/PMF_HTML/JsonPath_Expressions.htm) to a  list of values.  
For example, the following variations:

In [6]:
variations = [
{
        "a0" : [1,2,3,4],
        "a1.b0" : ['a','b','c','d']
},
{
        "a1.b1[2]" : ['x','y','z'],
        "a1.b1[0]" : [3,2,1]
},
]

Contains two groups. One with the paths a0 and a1.b0 and the other with the paths a1.b1[0],a1.b1[2].

The path a0 is the a0 from the root. The path a1.b0 is when b0 is the son of the a1. 
a1.b1[0] is the first term in the son b1 of a1 (given that b1 is a list). 

The JSONVariations creates variations of the cartesian product of all the values between the variation groups. 
parameters within the variation group change together. Hence, all the members of one variation group must an identical number of 
values. 

For example when the base file is 

In [7]:
base = {
 "a0" : 1,
 "a1" : {
    "b0" : 1,
    "b1" : [1,2,3],
     "b2" : {
             "c0": 0
     }
 }  
}

The variations of the variation file given above are: 

In [8]:
for x in JSONVariations(base,variations):
    print(x)

{'a0': 1, 'a1': {'b0': 'a', 'b1': [3, 2, 'x'], 'b2': {'c0': 0}}}
{'a0': 1, 'a1': {'b0': 'a', 'b1': [2, 2, 'y'], 'b2': {'c0': 0}}}
{'a0': 1, 'a1': {'b0': 'a', 'b1': [1, 2, 'z'], 'b2': {'c0': 0}}}
{'a0': 2, 'a1': {'b0': 'b', 'b1': [3, 2, 'x'], 'b2': {'c0': 0}}}
{'a0': 2, 'a1': {'b0': 'b', 'b1': [2, 2, 'y'], 'b2': {'c0': 0}}}
{'a0': 2, 'a1': {'b0': 'b', 'b1': [1, 2, 'z'], 'b2': {'c0': 0}}}
{'a0': 3, 'a1': {'b0': 'c', 'b1': [3, 2, 'x'], 'b2': {'c0': 0}}}
{'a0': 3, 'a1': {'b0': 'c', 'b1': [2, 2, 'y'], 'b2': {'c0': 0}}}
{'a0': 3, 'a1': {'b0': 'c', 'b1': [1, 2, 'z'], 'b2': {'c0': 0}}}
{'a0': 4, 'a1': {'b0': 'd', 'b1': [3, 2, 'x'], 'b2': {'c0': 0}}}
{'a0': 4, 'a1': {'b0': 'd', 'b1': [2, 2, 'y'], 'b2': {'c0': 0}}}
{'a0': 4, 'a1': {'b0': 'd', 'b1': [1, 2, 'z'], 'b2': {'c0': 0}}}
